In [ ]:
%matplotlib inline
import plot
import market
import metrics
from decimal import Decimal

In [ ]:
m = market.Returns_US_1871()
years = 30
start_year = 1871
last_year = 2015-years

def run(stock_pct):
    swrs = {}
    for _ in range(last_year - start_year + 1):
        returns = []
#        if start_year + _ in (1965,1966,1967,1968,1969,1970,1971,1972,1973): continue
        for one_year in zip(range(years), m.iter_from(start_year + _)):
            annual_returns = one_year[1]
            stocks = annual_returns.stocks - annual_returns.inflation
            bonds = annual_returns.bonds - annual_returns.inflation
            returns.append((stocks * stock_pct) + (bonds * (1-stock_pct)))
        swrs[start_year + _] = float(metrics.ssr(returns))
    return swrs

mswr = run(Decimal(.6))

In [ ]:
plot.plot([mswr[i] for i in mswr], y_label='30 year safe withdrawal rate', add_commas=False)

In [ ]:
plot.seaborn.distplot([mswr[i] for i in mswr])

In [ ]:
import pandas
import scipy.stats
pds = pandas.Series([mswr[i] for i in mswr])

scipy.stats.percentileofscore(pds, .05)
pds.quantile(.05)

In [ ]:
under45 = {k:v for k,v in mswr.items() if v <.045 }
s = pandas.Series()
for k,v in under45.items():
    s.loc[k] = v
print(s.head())
#s.to_csv('under45.csv')

In [ ]:
import metrics

def get_inflation(year, length=15):
    a = list(zip(range(length), m.iter_from(year)))
    inflations = [x[1].inflation for x in a]
    return metrics.average(inflations)

inflation = {}
for _ in range(last_year - start_year + 1):
    year = start_year + _
    inflation[year] = get_inflation(year)

In [ ]:
plot.plot_n({'MSWR' : [mswr[i] for i in sorted(mswr.keys())], 'Inflation' : [inflation[i] for i in sorted(inflation.keys())]},
           xlabel='Year', title='MSWR vs Inflation', add_commas=False)

In [ ]:
cor_frame = pandas.DataFrame({'MSWR' : [mswr[i] for i in sorted(mswr.keys())], 'Inflation' : [inflation[i] for i in sorted(inflation.keys())]})
from statsmodels.formula.api import ols
model = ols("MSWR ~ Inflation", cor_frame).fit()
print(model.summary())

In [ ]:
plot.seaborn.regplot(x='Inflation', y='MSWR', data=cor_frame)

In [ ]:
from statsmodels.formula.api import ols

def get_stocks(year, length=5):
    a = list(zip(range(length), m.iter_from(year)))
    stocks = [float(x[1].stocks) for x in a]
    return scipy.stats.gmean([x+1 for x in stocks]) - 1

def make_model(length):
    stocks = {}
    for _ in range(last_year - start_year + 1):
        year = start_year + _
        stocks[year] = get_stocks(year, length=length)
    cor_frame = pandas.DataFrame({'MSWR' : [mswr[i] for i in sorted(mswr.keys())], 'EquityReturns' : [stocks[i] for i in sorted(stocks.keys())]})
    model = ols("MSWR ~ EquityReturns", cor_frame).fit()
    return model

equity_check = pandas.Series()

for i in range(5):
    equity_check.loc[i] = make_model(i+1).rsquared

plot.seaborn.tsplot(equity_check)
#plot.seaborn.regplot(x='EquityReturns', y='MSWR', data=cor_frame)

In [ ]:
from statsmodels.formula.api import ols

def make_model(length):
    stocks = {}
    for _ in range(last_year - start_year + 1):
        year = start_year + _
        stocks[year] = get_inflation(year, length=length)
    cor_frame = pandas.DataFrame({'MSWR' : [mswr[i] for i in sorted(mswr.keys())], 'EquityReturns' : [stocks[i] for i in sorted(stocks.keys())]})
    model = ols("MSWR ~ EquityReturns", cor_frame).fit()
    return model

equity_check = pandas.Series()

for i in range(30):
    equity_check.loc[i] = make_model(i+1).rsquared

plot.seaborn.tsplot(equity_check)
#plot.seaborn.regplot(x='EquityReturns', y='MSWR', data=cor_frame)

In [ ]:
from statsmodels.formula.api import ols

def get_stocks(year, length=5):
    a = list(zip(range(length), m.iter_from(year)))
    stocks = [float(x[1].bonds) - float(x[1].inflation) for x in a]
    return scipy.stats.gmean([x+1 for x in stocks]) - 1

def make_model(length):
    stocks = {}
    for _ in range(last_year - start_year + 1):
        year = start_year + _
        stocks[year] = get_stocks(year, length=length)
    cor_frame = pandas.DataFrame({'MSWR' : [mswr[i] for i in sorted(mswr.keys())], 'EquityReturns' : [stocks[i] for i in sorted(stocks.keys())]})
    model = ols("MSWR ~ EquityReturns", cor_frame).fit()
    return model

equity_check = pandas.Series()

for i in range(30):
    equity_check.loc[i] = make_model(i+1).rsquared

print(equity_check)
plot.seaborn.tsplot(equity_check)
#plot.seaborn.regplot(x='EquityReturns', y='MSWR', data=cor_frame)